# Building a Student Intervention System

This is a classification problem because the target column is made up of categorical data. If it will have been a regression problem if the target column had been made up of real / continious values.

## Exploring the Data

Let's go ahead and read in the student dataset first.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]-1
n_passed = student_data.passed[student_data.passed == 'yes'].count()
n_failed = student_data.passed[student_data.passed == 'no'].count()
grad_rate = (float(n_passed) / n_students) * 100.0
print("Total number of students: {}".format(n_students))
print("Number of students who passed: {}".format(n_passed))
print("Number of students who failed: {}".format(n_failed))
print("Number of features: {}".format(n_features))
print("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print("Feature column(s):-\n{}".format(feature_cols))
print("Target column: {}".format(target_col))

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print("\nFeature values:-")
print(X_all.head())  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print("Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
from sklearn.model_selection import train_test_split
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=300)
print("Training set: {} samples".format(X_train.shape[0]))
print("Test set: {} samples".format(X_test.shape[0]))
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


C:\Users\Tony Stark\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print("Done!\nTraining time (secs): {:.3f}".format(end - start))
    return end - start

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.005


0.004538297653198242

In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print("Done!\nPrediction time (secs): {:.3f}".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes'), end - start

train_f1_score = predict_labels(clf, X_train, y_train)
print("F1 score for training set: {}".format(train_f1_score[0]))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 1.0


In [9]:
# Predict on test data
print("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[0]))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.6818181818181818


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print("------------------------------------------")
    print("Training set size: {}".format(len(X_train)))
    train_classifier(clf, X_train, y_train)
    print("F1 score for training set: {}".format(predict_labels(clf, X_train, y_train)[0]))
    print("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[0]))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [11]:
# TODO: Train and predict using two other models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
classifiers = [DecisionTreeClassifier(), SVC(), GaussianNB()]
results = { 
        'Classifier': [],
        'Size': [], 
        'Train time': [], 
        'predict time': [], 
        'F1 score - train': [], 
        'F1 score - test': []
    }
datasets = [train_test_split(X_all, y_all, train_size=x, test_size=95) for x in [100, 200, 300]]
for clf in classifiers:
    for data in datasets:
        X_train, X_test, y_train, y_test = data
        time_train = train_classifier(clf, X_train, y_train)
        f1_train, time_predict = predict_labels(clf, X_train, y_train)
        f1_test, time_predict = predict_labels(clf, X_test,y_test)
        
        results['Classifier'].append(clf.__class__.__name__)
        results['Size'].append(X_train.shape[0])
        results['Train time'].append("{:.3f}".format(time_train))
        results['predict time'].append("{:.3f}".format(time_predict))
        results['F1 score - train'].append(f1_train)
        results['F1 score - test'].append(f1_test)
        
pd.DataFrame(results)

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.004
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.007
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Training SVC...
Done!
Training time (secs): 0.017
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
Training SVC...
Done!
Training time (secs): 0.004
Predicting labels using SVC...
Done!
Predic

C:\Users\Tony Stark\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Tony Stark\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Tony Stark\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


,Classifier,Size,Train time,predict time,F1 score - train,F1 score - test
0,DecisionTreeClassifier,100,0.003,0.001,1.000000,0.672000
1,DecisionTreeClassifier,200,0.000,0.000,1.000000,0.785185
2,DecisionTreeClassifier,300,0.007,0.000,1.000000,0.688525
3,SVC,100,0.017,0.002,0.884615,0.778523
4,SVC,200,0.004,0.006,0.891803,0.812903
5,SVC,300,0.008,0.003,0.869936,0.750000
6,GaussianNB,100,0.002,0.001,0.372093,0.494382
7,GaussianNB,200,0.004,0.000,0.792982,0.773723
8,GaussianNB,300,0.000,0.001,0.815348,0.655462


## Choosing the Best Model


### Reviewing the experiment's result
Based on my experiments carried out previously, I believe DecisionTree Classifiers is generally more appropriate based on the available data. To understand why, I will review the three models here in terms of Performance, Train time and Prediction Time. Remember, that one of our goals here is to use as little computation as possible.

#### Decision Trees
The average F1 Score `0.716` across all training sizes. The training time increased linearly with the training size and the prediction time is negligible.
#### Support Vector Classifiers.
The average F1 Score `0.826` across all training sizes. The training time increased exponentially with the training size and so does the prediction time. This model does have a better f1 score but it takes the most amount of time to both train and predict
#### Gaussian Naive Bayes
The average F1 Score `0.59` across all training sizes. The training time has no clear pattern relative to the training size, however, it is considerable low. The prediction time appears near constant, but it is more than the decision tree.

I would argue that the decision tree classifier is a better model for our data because it gives a good f1 score, get trained fast and also predicts data in negligible time.

### Explaining the Final Model
My Model's final f1 score is 0.8102
To do arrive at that score, gridsearch showed that setting a maximum depth to 2 nodes and maximum features using the square root of features produces the best model f1 score.

In [12]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
def performance_metric(label, prediction):
    return f1_score(label, prediction, pos_label='yes')

X_train, X_test, y_train, y_test = datasets[2]
parameters = {
    'max_features':['log2', 'sqrt'], 
    'max_depth':[1,2,3]}

# parameters = {}
# gnb = GaussianNB()
dtc = DecisionTreeClassifier()
clf = GridSearchCV(dtc, parameters, scoring = make_scorer(performance_metric, greater_is_better=True))
clf.fit(X_train, y_train)
f1_test, time_predict = predict_labels(clf.best_estimator_, X_test,y_test)
f1_test

C:\Users\Tony Stark\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.006


0.7313432835820896

In [13]:
clf.best_score_

0.8113820176016073